In [21]:
from music21 import *
import os
from fractions import *
import numpy as np
from midi_note import MIDINote
import json

In [22]:
#Global Variables

PATH = "POP909/"
LIMIT = 10
MIDI_NOTE_DIRECTORY = "midi_notes_list.json"
MAX_NOTE_VAL = 125



In [23]:
def parse_raw_notes(file):
    print("Loading Music File:", file)
    raw_notes = []

    midi_data = converter.parse(file)

    for part in midi_data.parts:
        if part.partName == 'MELODY':
            midi_elements = part.recurse()
            for element in midi_elements:
                if isinstance(element, note.Note) or isinstance(element, note.Rest):
                    note_duration = duration.Duration()
                    note_duration.quarterLength = element.quarterLength

                    if isinstance(element, note.Rest):
                        raw_note = str(element.name)
                    else:
                        raw_note = str(element.pitch)

                    raw_notes.append(MIDINote(raw_note, str(note_duration.type), str(element.quarterLength)))

    return np.array(raw_notes)

In [24]:
def get_midi_file_paths():
    path = PATH
    limit = LIMIT
    midi_files_path = []
    for index, i in enumerate(os.listdir(path)):
        if index == limit:
            break
        for j in os.listdir(path + i):
            midi_path = (i + '/' + j)
            if midi_path.endswith(".mid"):
                midi_files_path.append(midi_path)
    return np.array(midi_files_path)

In [25]:
def load_data():
    midi_files_path = get_midi_file_paths()

    raw_notes_list = []

    for midi_path in midi_files_path:
        parsed_midi_notes = parse_raw_notes(PATH + midi_path)
        raw_notes_list.append(parsed_midi_notes)

    return np.array(raw_notes_list).reshape((LIMIT, 1))

In [26]:
def to_json(midi_notes_list):
    json_file = {}
    for index, midi_notes in enumerate(midi_notes_list):
        notes = midi_notes[0]

        json_file[index] = [_note.as_map for _note in notes]

    with open(MIDI_NOTE_DIRECTORY) as f:
        data = json.load(f)
        data.update(json_file)
    with open(MIDI_NOTE_DIRECTORY, 'w') as f:
        json.dump(data, f)

In [27]:
def json_to_midi_notes_list(batch=None):
    """Converts json to numpy array
    Args
        batch: A list/tuple defining the start and end to be converted
    """

    with open(MIDI_NOTE_DIRECTORY) as f:
        data = json.load(f)
    m = len(data)
    midi_notes_list = []

    if batch:
        start, end = batch
        end = min(m-1, end)
        for index in range(start, end+1):
            midi_notes_list.append(np.array(data[str(index)]))
    else:
        for index in range(m):
            midi_notes_list.append(np.array(data[str(index)]))

    return np.array(midi_notes_list)

In [43]:
def map_note(_note, duration):
    """Adjusts note strings to be between the 5th & 6th octaves,
       \nthen maps the adjusted note string to an integer.

       Parameters:
         _note (string): A string representing a note.

       Returns:
        int: An integer representing the note.
    """
    duration_map = {
        'whole': 0,
        'half': 1,
        'quarter': 2,
        'eighth': 3,
        '16th': 4,
    }

    # TODO: Round off to nearest duration
    duration_conversion_map = {
        '32nd': '16th',
        'complex': 'whole',
    }
    try:
        duration = duration_conversion_map[duration]
    except:
        pass

    if (_note == 'rest'):
        return 120 + duration_map[duration]

    note_letter = _note[:-1]
    note_octave = _note[-1]
    # TODO: Prevent bias against 5th octave
    try:
        note_octave = int(note_octave)
    except:
        note_letter = _note
        note_octave = 4
    note_octave = 4 if note_octave <= 4 else 5
    flats_map = {
        'D-': 'C#',
        'E-': 'D#',
        'G-': 'F#',
        'A-': 'G#',
        'B-': 'A#'
    }
    try:
        note_letter = flats_map[note_letter]
    except:
        pass
    notes_map = {
        'C': 0,
        'C#': 5,
        'D': 10,
        'D#': 15,
        'E': 20,
        'F': 25,
        'F#': 30,
        'G': 35,
        'G#': 40,
        'A': 45,
        'A#': 50,
        'B': 55,
    }
    print(note_letter, note_octave)
    print(notes_map[note_letter], duration_map[duration], ((note_octave-4) * 60))
    return notes_map[note_letter] + duration_map[duration] + ((note_octave-4) * 60)

def normalize(mapped_notes):
    """Normalize data using min-max scaling
    Args
        value: Integer from 0-125 note mapping

    """
    return mapped_notes/MAX_NOTE_VAL

def transform_dataset(all_midi_notes_list):
    """Transforms numpy array of midi notes list into model input data for training
    Args
        all_midi_notes_list: list of all midi notes
    """

    dataset = []

    for notes_list in all_midi_notes_list:
        mapped = []
        for _note in notes_list:
            mapped.append(map_note(_note['note'], _note['duration_type']))

        dataset.append(normalize(np.array(mapped)))

    return np.array(dataset)




In [29]:
# Runner
all_midi_notes = load_data()
to_json(all_midi_notes)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'POP909/'

In [31]:
test = json_to_midi_notes_list(batch=[0,0])
test

array([[{'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'half', 'length': '2.0'},
        {'note': 'rest', 'duration_type': 'quarter', 'length': '1.0'},
        {'note': 'C#4', 'duration_type': '16th', 'length': '0.25'},
        {'note': 'rest', 'duration_type': '32nd', 'length': '1/12'},
        {'note': 'E-4', 'duration_type': '16th', 'length': '0.25'},
        {'note': 'F#4', 'duration_type': '16th', 'length': '0.25'},
        {'note': 'G#4', 'duration_type': '16

In [45]:
test2 = transform_dataset(test)
test2

C# 4
5 4 0
D# 4
15 4 0
F# 4
30 4 0
G# 4
40 4 0
A# 4
50 4 0
F# 4
30 4 0
D# 4
15 4 0
G# 4
40 3 0
G# 4
40 2 0
G# 4
40 4 0
F 4
25 4 0
C# 4
5 4 0
F# 4
30 3 0
F# 4
30 3 0
C# 4
5 4 0
D# 4
15 4 0
F# 4
30 4 0
G# 4
40 4 0
A# 4
50 4 0
F# 4
30 4 0
D# 4
15 4 0
G# 4
40 3 0
G# 4
40 4 0
C# 4
5 4 0
G# 4
40 4 0
F# 4
30 3 0
F# 4
30 0 0
F# 4
30 2 0
F# 4
30 3 0
F 4
25 4 0
F# 4
30 4 0
F 4
25 3 0
F 4
25 4 0
F# 4
30 4 0
F 4
25 4 0
C# 4
5 4 0
D# 4
15 4 0
D# 4
15 3 0
D# 4
15 4 0
F 4
25 4 0
F# 4
30 3 0
F# 4
30 2 0
F 4
25 4 0
F# 4
30 4 0
F 4
25 4 0
D# 4
15 4 0
C# 4
5 4 0
C# 4
5 2 0
F# 4
30 3 0
F# 4
30 2 0
G# 4
40 4 0
A# 4
50 4 0
G# 4
40 4 0
G# 4
40 4 0
A# 4
50 4 0
G# 4
40 4 0
F 4
25 4 0
F# 4
30 4 0
F# 4
30 1 0
C# 4
5 4 0
D# 4
15 4 0
F# 4
30 4 0
A# 4
50 4 0
G# 4
40 4 0
F# 4
30 4 0
G# 4
40 4 0
A# 4
50 4 0
G# 4
40 4 0
F# 4
30 4 0
F# 4
30 1 0
C# 4
5 4 0
D# 4
15 4 0
F# 4
30 4 0
G# 4
40 4 0
A# 4
50 4 0
F# 4
30 4 0
D# 4
15 4 0
G# 4
40 3 0
G# 4
40 2 0
G# 4
40 4 0
F 4
25 4 0
C# 4
5 4 0
F# 4
30 3 0
F# 4
30 3 0
C# 4
5 4 0
D

array([[0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968, 0.968,
        0.976, 0.072, 0.992, 0.152, 0.272, 0.352, 0.432, 0.992, 0.272,
        0.992, 0.152, 0.992, 0.344, 0.336, 0.976, 0.352, 0.992, 0.232,
        0.992, 0.072, 0.992, 0.264, 0.264, 0.992, 0.072, 0.992, 0.152,
        0.272, 0.352, 0.432, 0.992, 0.272, 0.992, 0.152, 0.992, 0.344,
        0.352, 0.992, 0.072, 0.992, 0.352, 0.992, 0.264, 0.24 , 0.984,
        0.968, 0.256, 0.984, 0.264, 0.984, 0.232, 0.992, 0.272, 0.224,
        0.976, 0.232, 0.992, 0.272, 0.232, 0.984, 0.072, 0.992, 0.152,
        0.144, 0.984, 0.152, 0.232, 0.264, 0.992, 0.256, 0.984, 0.232,
        0.992, 0.272, 0.232, 0.984, 0.152, 0.992, 0.072, 0.056, 0.992,
        0.968, 0.264, 0.984, 0.256, 0.984, 0.352, 0.992, 0.432, 0.352,
        0.976, 0.352, 0.992, 0.432, 0.352, 0.984, 0.232, 0.992, 0.272,
        0.248, 0.984, 0.072, 0.152, 0.272, 0.992, 0.432, 0.352, 0.984,
        0.272, 0.984, 0.352, 0.992, 0.432, 0.352, 0.984, 0.272, 0.984,
      

NameError: name 'rand' is not defined